# Subchallenge 1

### Ian Ren,  Shang Wang, Xiling Li

We'll train a RidgeRegression model on RNASeq data for each inhibitor to predict AUC. Specifically:

- Select the n highest variance genes
- For each inhibitor, train with leave-one-out cross validation
- Try a bunch of alpha values

###  Load the data

In [73]:
import pandas
# data 
rnaseq = pandas.read_csv('training_real/rnaseq_real.csv')
# gt
aucs = pandas.read_csv('training_real/aucs.csv')

### leadboard data

In [22]:
# data 
rnaseq_lb = pandas.read_csv('training/rnaseq.csv')
# gt
aucs_lb = pandas.read_csv('training/aucs.csv')

In [29]:
#rnaseq_combine = pandas.merge(rnaseq,rnaseq_lb)
rnaseq_combine = pandas.read_csv('training_combine/rnaseq_combine.csv')
aucs_combine = pandas.read_csv('training_combine/aucs_combine.csv')

In [74]:
import joblib
from util import TransposeRnaSeqTable
# Select the 2400 highest variance genes
specimens = TransposeRnaSeqTable(rnaseq)
#specimens_combine = TransposeRnaSeqTable(rnaseq_combine)

### Here we proposed 5 feature selection methods on genes

### 1. Manually selected genes
- published from paper "A machine learning approach to integrate big data for precision medicine in acute myeloid leukemia"
- Lee, S., Celik, S., Logsdon, B.A. et al. A machine learning approach to integrate big data for precision medicine in acute myeloid leukemia. Nat Commun 9, 42 (2018). https://doi.org/10.1038/s41467-017-02465-5

In [2]:
naturecommm2018_symbol = ['FLT3', 'CASP8AP2', 'L2HGDH', 'MNT', 'BAZ2B', 'MZF1', 'BEX2', 'SMARCA4']
naturecommm2018_gene = []
for _,row in rnaseq.iterrows():
    if row.Symbol in naturecommm2018_symbol:
        print(row.Gene)
        naturecommm2018_gene.append(row.Gene)

ENSG00000070444
ENSG00000087299
ENSG00000099326
ENSG00000118412
ENSG00000122025
ENSG00000123636
ENSG00000127616
ENSG00000133134


### 2. Select the n highest variance genes
- Through our experiments, we can get the best result when we take n = 25000

In [50]:
selected_2400_genes = specimens.var().nlargest(2400).index.tolist()

In [114]:
selected_n_genes = specimens.var().nlargest(1000).index.tolist()

### Then, after we conduct heatmap on gene mutation data("dnaseq.csv"),  we divide the 238 genes into 4 groups based on the number of mutations (few, medial, many, too many), and we only take "few", "medial" and "many" groups as selection results.

### 3. The gene group with few mutations 

In [5]:
few_mutations_symbols = ['DIAPH1', 'C17orf97', 'UNC80', 'NUP214', 'JMJD1C', 'AHDC1', 'FAM205A', 'LRRC37A3', 
                    'CD109', 'NUP98', 'CHIA', 'KCNK1', 'BMPER', 'CNTN6', 'PLIN4', 'EPHA3', 'ACAN', 'ABI3BP', 
                    'PKP3', 'PLEKHG5', 'ZNF285', 'TLR10', 'THADA', 'GLB1L2', 'CRY1', 'C1QC', 'RPGR', 'NMD3', 
                    'DSCAM', 'CDH20', 'NLRP1', 'SP140L', 'TLL2', 'BHLHE22', 'PPFIA3', 'MYO10', 'HEATR1', 
                    'KRT5', 'EP300', 'MORC4', 'KIAA1210', 'MXRA5', 'WDR87', 'TRPM2', 'TRANK1', 'IGFN1', 
                    'TMPRSS6', 'PHF6', 'GRIK4', 'ENAH', 'ADAMTS7', 'TET2', 'RBMS1', 'HMGCLL1', 'KIAA1239', 
                    'SCN11A', 'MYO16', 'CNTNAP3', 'ADNP2', 'FLNB', 'MYH13', 'GRM4', 'NACAD', 'HUNK', 'ARMC4', 
                    'CREBBP', 'ZNF711', 'A1CF', 'TRPM3', 'IKZF1', 'SOBP', 'NLRP8', 'SPOCK2', 'KAT8', 'CHCHD10', 
                    'TRMT5', 'WNK1', 'TP53', 'KRT10', 'SDHA', 'IVL', 'SPEG', 'SF3B1', 'TMC3', 'ACTL6B', 'WT1', 'FAM186A', 
                    'RBM12B', 'AQR', 'JAK2', 'DNMT3A', 'NPM1', 'TMEM175', 'ZNF195', 'CELSR2', 'IDH2', 'NRAS', 'COG7', 
                    'PCNXL3', 'KRAS', 'SCAF1', 'CDK13', 'SPEF2', 'YIPF5', 'FLT3', 'TPTE2']

few_mutations_genes = []
for _,row in rnaseq.iterrows():
    if row.Symbol in few_mutations_symbols:
        few_mutations_genes.append(row.Gene)
print('Length is :', len(few_mutations_genes))

('Length is :', 107)


### 4. The gene group with medial mutations 

In [46]:
medial_mutations_symbols = ['IRX5', 'FNDC3B', 'TRDN', 'CHD4', 'ACSM2A', 'ABCA9', 'OTOG', 'STK3', 'ADCY8', 'CRB1', 
                         'WDR43', 'SUSD5', 'P2RY2', 'KRTAP5-7', 'TENM2', 'XIRP1', 'CYP4F12', 'PAPPA2', 
                         'CEACAM6', 'SASH1', 'TCHH', 'MARK3', 'TENM3', 'ASXL1', 'KCNG4', 'IRX6', 'CDH4', 
                         'DDX60L', 'RIMBP3', 'ADAMTS12', 'KIAA1328', 'ELFN1', 'AARS2', 'LRP8', 'SMC3', 'NCAM2', 
                         'PTPRS', 'F5', 'RUNX1', 'PLRG1', 'SH3KBP1', 'ATXN3', 'NF1', 'ATXN1', 'CTNNA2', 'PTEN', 
                         'KAT6B', 'TRIM48', 'IMPG1', 'FGD6', 'PLEKHG2', 'SMC1A', 'FOXP1', 'ARMC10', 'SLC39A12', 
                         'ZBTB47', 'NPIPA5', 'TRIO', 'FABP2', 'SSC5D', 'CT47B1', 'FAM194A', 'CDON', 'HECW2']

medial_mutations_genes = []
for _,row in rnaseq.iterrows():
    if row.Symbol in medial_mutations_symbols:
        medial_mutations_genes.append(row.Gene)
print('Length is :', len(medial_mutations_genes))

('Length is :', 67)


### 5. The gene group with many mutations 

In [ ]:
many_mutations_symbols = ['IRX5', 'FNDC3B', 'TRDN', 'CHD4', 'ACSM2A', 'ABCA9', 'OTOG', 'STK3', 'ADCY8', 'CRB1', 
                         'WDR43', 'SUSD5', 'P2RY2', 'KRTAP5-7', 'TENM2', 'XIRP1', 'CYP4F12', 'PAPPA2', 
                         'CEACAM6', 'SASH1', 'TCHH', 'MARK3', 'TENM3', 'ASXL1', 'KCNG4', 'IRX6', 'CDH4', 
                         'DDX60L', 'RIMBP3', 'ADAMTS12', 'KIAA1328', 'ELFN1', 'AARS2', 'LRP8', 'SMC3', 'NCAM2', 
                         'PTPRS', 'F5', 'RUNX1', 'PLRG1', 'SH3KBP1', 'ATXN3', 'NF1', 'ATXN1', 'CTNNA2', 'PTEN', 
                         'KAT6B', 'TRIM48', 'IMPG1', 'FGD6', 'PLEKHG2', 'SMC1A', 'FOXP1', 'ARMC10', 'SLC39A12', 
                         'ZBTB47', 'NPIPA5', 'TRIO', 'FABP2', 'SSC5D', 'CT47B1', 'FAM194A', 'CDON', 'HECW2']

many_mutations_genes = []
for _,row in rnaseq.iterrows():
    if row.Symbol in many_mutations_symbols:
        many_mutations_genes.append(row.Gene)
print('Length is :', len(many_mutations_genes))

### 6. highly significant genes from t-test of DNAseq data

In [25]:
ttest_symbols_1 = ['TTC40', 'TTLL1', 'FAM160A1', 'MTNR1B', 'STAG2', 'IL12RB1', 'CASS4', 'PDS5B', 'LPIN3', 
               'SEMA5A', 'POM121', 'IDH1', 'B4GALT2', 'ZZEF1', 'BCOR', 'TTI1', 'NRXN3', 'TCF25', 'FAF1', 
               'CCDC157', 'TPRX1', 'TRIP12', 'AGTPBP1', 'ZFHX4', 'LRP4', 'TTBK1', 'PKD1L2', 'SRSF2', 'SGK223', 
               'TUBA3D', 'TNIK', 'SLC9B1', 'CHD3', 'U2AF1', 'SP4', 'KIT', 'LRRCC1', 'ZNF518B', 'MTUS2', 'FAM83C', 
               'TRIM77', 'TRIM64B', 'EFCC1', 'ANAPC1', 'MYO5B', 'DARC', 'ARID2', 'CYP2A13', 'SH3TC2', 'REV3L']
ttest_genes_1 = []
for _,row in rnaseq.iterrows():
    if row.Symbol in ttest_symbols_1:
        ttest_genes_1.append(row.Gene)
print('Length is :', len(ttest_genes_1))

('Length is :', 51)


### 7. highly significant genes from t-test of DNAseq data with RNAseq data 

In [6]:
ttest_symbols_2 = ['AGTPBP1', 'SMC1A', 'IDH1', 'PLRG1', 'ASXL1', 'KIT', 'RIMBP3', 'NPIPA5', 'OTOG', 'BCOR', 
                   'ELFN1', 'CHD4', 'TRIM77', 'CACNA2D3', 'TENM3', 'ANAPC1', 'PPP1R21', 'IRX5', 'B4GALT2', 
                   'MYO5B', 'DARC', 'ASCC3', 'ARID2', 'FER1L6', 'POM121', 'CYP2A13', 'CYP4F12', 'PKD1L2', 
                   'KIAA1328', 'FAF1', 'KCNG4', 'DTX1', 'CHD3', 'PEAR1', 'CCDC157']
ttest_genes_2 = []
for _,row in rnaseq.iterrows():
    if row.Symbol in ttest_symbols_2:
        ttest_genes_2.append(row.Gene)
print('Length is :', len(ttest_genes_2))

('Length is :', 37)


## 2 methods for combine feature selection result
- 1. combain different groups gene
- 2. train different models for different groups

In [115]:
# combine fearure
#combined_genes = list(set(naturecommm2018_gene+selected_n_genes+few_mutations_genes+ttest_genes_2))
test_genes = list(selected_n_genes)

In [116]:
len(test_genes)

1000

### Ridgecv

In [82]:
from matplotlib import pyplot
import numpy
import seaborn
from sklearn.linear_model import RidgeCV

# Normalize each specimen.
X = specimens
X = X.div(numpy.linalg.norm(X, axis=1), axis=0)
X = X[test_genes]

# Compute z-score. normal?
gene_mean = X.mean(axis=0)
gene_std = X.std(axis=0)
X = (X - gene_mean) / gene_std

# For each inhibitor, train a regressor.
alphas = numpy.logspace(-1, 5, num=40) # alpha
regressors = {}
for inhibitor in aucs.inhibitor.unique():
    auc_subset = aucs[aucs.inhibitor == inhibitor]
    regr = RidgeCV(alphas=alphas, store_cv_values=True) #
    regr = regr.fit(X.loc[auc_subset.lab_id], auc_subset.auc)  
    regressors[inhibitor] = regr

### Ridgecv on combine data

In [79]:
from matplotlib import pyplot
import numpy
import seaborn
from sklearn.linear_model import RidgeCV

# Normalize each specimen.
X = specimens_combine
X = X.div(numpy.linalg.norm(X, axis=1), axis=0)
X = X[test_genes]

# Compute z-score. normal?
gene_mean = X.mean(axis=0)
gene_std = X.std(axis=0)
X = (X - gene_mean) / gene_std

# For each inhibitor, train a regressor.
alphas = numpy.logspace(-1, 5, num=40) # alpha
regressors = {}
for inhibitor in aucs_combine.inhibitor.unique():
    auc_subset = aucs_combine[aucs_combine.inhibitor == inhibitor]
    regr = RidgeCV(alphas=alphas, store_cv_values=True) #
    regr = regr.fit(X.loc[auc_subset.lab_id], auc_subset.auc)  
    regressors[inhibitor] = regr

AttributeError: 'float' object has no attribute 'sqrt'

### LS

In [117]:
from matplotlib import pyplot
import numpy
import seaborn
from sklearn.linear_model import RidgeCV

# Normalize each specimen.
X = specimens
X = X.div(numpy.linalg.norm(X, axis=1), axis=0)
X = X[test_genes]

# Compute z-score. normal?
gene_mean = X.mean(axis=0)
gene_std = X.std(axis=0)
X = (X - gene_mean) / gene_std

# For each inhibitor, train a regressor.
#alphas = numpy.logspace(-1, 5, num=40) # alpha
regressors = {}
for inhibitor in aucs.inhibitor.unique():
    auc_subset = aucs[aucs.inhibitor == inhibitor]
    #regr = RidgeCV(alphas=alphas, store_cv_values=True) #
    regr = LassoCV(alphas=alphas)
    regr = regr.fit(X.loc[auc_subset.lab_id], auc_subset.auc)  
    regressors[inhibitor] = regr

### Store the model information in `model/`

In python this process is sometimes referred to as "pickling," thus the variable names "pkl_1" and "pkl_2".

In [108]:
pkl_1_dict = {
    'gene': test_genes,
    'gene_mean': gene_mean,
    'gene_std': gene_std,
}
for inhibitor, regr in regressors.items():
    pkl_1_dict[inhibitor] = regr.coef_
pkl_1_out = pandas.DataFrame(pkl_1_dict)

pkl_2_out = pandas.DataFrame({
    'inhibitor': [inhibitor for inhibitor in regressors.keys()],
    'intercept': [regr.intercept_ for regr in regressors.values()],
})
pkl_1_out.to_csv('model/pkl_1.csv', index=False)
pkl_2_out.to_csv('model/pkl_2.csv', index=False)

### Run the model

We run the model with

```bash
SYNAPSE_PROJECT_ID=<your project ID> #syn21555820
docker build -t docker.synapse.org/$SYNAPSE_PROJECT_ID/sc1_model .
docker run \
    -v "$PWD/training/:/input/" \
    -v "$PWD/output:/output/" \
    docker.synapse.org/$SYNAPSE_PROJECT_ID/sc1_model

# Maybe push to Synapse.
docker login docker.synapse.org
docker push docker.synapse.org/$SYNAPSE_PROJECT_ID/sc1_model
```
**Note: you must pass the Synapse [certified user test](https://docs.synapse.org/articles/accounts_certified_users_and_profile_validation.html#certified-users) before you can push to the Synapse docker hub.**
### Look at predictions vs goldstandard for training data

Assumes predictions are in `output/predictions.csv`. Note that the performance on training dataset is going to be better than on the leaderboard data. Therefore, this is a good test of formatting / sanity check, but not of predictive performance.

In [334]:
# Join groundtruth onto predictions, because predictions is a
# superset of groundtruth.
indices = ['lab_id', 'inhibitor']
groundtruth = pandas.read_csv('training/aucs.csv').set_index(indices)
predictions = pandas.read_csv('output/predictions.csv').set_index(indices)
predictions_and_groundtruth = groundtruth.join(
    predictions, lsuffix='_groundtruth', rsuffix='_prediction')

## Here, we test our model using leadborad data
### The naming does not change, but we change the data inside the floder

In [121]:
# test for result
from scipy import stats
from numpy import *

groundtruth_real = pandas.read_csv('training/aucs.csv').set_index('inhibitor')
predictions_real = pandas.read_csv('output/predictions.csv').set_index('inhibitor')

In [122]:
result_spearman = []
result_pearson = []

inhibitors = set(pandas.read_csv('training/aucs.csv').inhibitor)
for inhibitor in inhibitors:
    tmp = groundtruth_real.loc[inhibitor].set_index('lab_id').join(
                predictions_real.loc[inhibitor].set_index('lab_id'), 
                lsuffix='_groundtruth', rsuffix='_prediction')
    result_spearman.append(stats.spearmanr(tmp['auc_groundtruth'],tmp['auc_prediction'])[0])
    result_pearson.append(stats.pearsonr(tmp['auc_groundtruth'],tmp['auc_prediction'])[0])

In [123]:
print(mean(result_spearman),mean(result_pearson))

(nan, nan)


In [128]:
#r = [x for x in result_pearson if str(x) != 'nan']
#mean(r)

### model ensemble

In [8]:
import pandas
predictions1 = pandas.read_csv('output/predictions.csv')
predictions2 = pandas.read_csv('output/predictions_2400.csv')
#predictions3 = pandas.read_csv('output/predictions_few.csv')
#predictions4 = pandas.read_csv('output/predictions_medial.csv')
#predictions5 = pandas.read_csv('output/predictions_many.csv')

In [327]:
array = []
for i in range(0,9760):
    array.append(float(predictions1.loc[[i],:]['auc']*0.2 + predictions2.loc[[i],:]['auc']*0.4 \
                 + predictions3.loc[[i],:]['auc']*0.2 + predictions4.loc[[i],:]['auc']*0.1 \
                 + predictions5.loc[[i],:]['auc']*0.1))
    

In [15]:
#predictions1

In [20]:
predictions1['auc'] = predictions1['auc']*0.5 + predictions2['auc']*0.5

In [23]:
#predictions1
#predictions1['auc']*0.5 + predictions2['auc']*0.5
predictions1['auc'] /= 3

In [24]:
predictions1['auc']

0       91.164920
1       90.962320
2       90.445791
3       91.120758
4       90.572853
5       90.590045
6       90.360094
7       92.256475
8       90.231167
9       90.682610
10      89.763036
11      92.154531
12      91.229698
13      88.005392
14      90.681908
15      89.042089
16      85.779689
17      91.724034
18      92.740854
19      91.560889
20      91.460592
21      89.779850
22      91.459511
23      91.559877
24      90.763487
25      92.702774
26      91.506807
27      90.424799
28      91.505797
29      90.459564
          ...    
9730    47.873852
9731    41.206725
9732    43.538882
9733    51.131223
9734    39.331104
9735    46.421430
9736    46.251611
9737    42.645018
9738    42.798965
9739    43.014878
9740    39.027962
9741    48.946949
9742    39.296324
9743    51.111952
9744    41.562367
9745    52.156057
9746    41.599107
9747    45.428982
9748    43.869547
9749    44.000673
9750    46.539728
9751    42.451316
9752    49.582579
9753    41.501891
9754    52

In [25]:
predictions1

,inhibitor,lab_id,auc
0,Vismodegib (GDC-0449),13-00163,91.164920
1,Vismodegib (GDC-0449),13-00232,90.962320
2,Vismodegib (GDC-0449),13-00365,90.445791
3,Vismodegib (GDC-0449),13-00393,91.120758
4,Vismodegib (GDC-0449),13-00454,90.572853
5,Vismodegib (GDC-0449),13-00551,90.590045
6,Vismodegib (GDC-0449),13-00552,90.360094
7,Vismodegib (GDC-0449),13-00601,92.256475
8,Vismodegib (GDC-0449),14-00015,90.231167
9,Vismodegib (GDC-0449),14-00240,90.682610
